In [26]:
# !pip install httpx
import matplotlib
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
matplotlib.rcParams['font.sans-serif'] = "KaiTi"
from datetime import date, datetime

## 网易接口

网易接口获取历史每日数据

URL 格式：


```
http://quotes.money.163.com/service/chddata.html?code=代码&start=开始时间&end=结束时间&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;TURNOVER;VOTURNOVER;VATURNOVER;TCAP;MCAP
```


**参数说明**：代码为股票代码，上海股票前加`0`，如 600756 变成 0600756，深圳股票前加`1`。时间都是6位标识法，年月日，fields标识想要请求的数据。可以不变。

例如大盘指数数据查询（上证指数 000001 前加 `0`，沪深300指数 000300 股票前加 `0`，深证成指 399001 前加 `1`，中小板指399005前加`1`，创业板指 399006 前加`1`）：

```
http://quotes.money.163.com/service/chddata.html?code=0000300&start=20151219&end=20171108&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;VOTURNOVER
```


In [35]:
import httpx
import pandas as pd
from io import StringIO 

def fetch_data(code, start_date, end_date):
    url = f'http://quotes.money.163.com/service/chddata.html?code={code}&start={start_date}&end={end_date}&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;TURNOVER;VOTURNOVER;VATURNOVER;TCAP;MCAP'
    print(f'url: {url}')
    r = httpx.get(url)
    r.encoding = 'gb18030'
    data = pd.read_csv(StringIO(r.text), index_col=0, parse_dates=[0])
    return data

@interact(code="0600036", start_time=widgets.DatePicker(description='start date'), end_time=widgets.DatePicker(description='end date'))
def f(code, start_time, end_time):
    if start_time is None:
        start_time = date(2022, 1, 1)
    if end_time is None:
        end_time = date.today()
    start_date = start_time.strftime("%Y%m%d")
    end_date = end_time.strftime("%Y%m%d")
    result = fetch_data(code, start_date, end_date)
    return result


interactive(children=(Text(value='0600036', description='code'), DatePicker(value=None, description='start dat…

## Souhu 数据接口

搜狐数据接口：

http://q.stock.sohu.com/hisHq?code={code}&start={start}&end={end}，{code}替换为股票代码，大陆股票代码加前缀cn_。{start}替换为起始日期，{end}替换为结束日期。


例如：在浏览器地址栏输入：

http://q.stock.sohu.com/hisHq?code=cn_601766,cn_000002&start=20150601&end=20160414

数组中数据一次对应含义如下：

日期,
今日开盘价,
今日收盘价,
涨跌幅度，
涨跌百分比,
今日最低价,
今日最高价，
今日成交股票手数,
今日成交金额(单位：万)，
换手率

In [4]:
import json
url = "https://q.stock.sohu.com/hisHq?code=cn_600036&start=20220101&end=20220723"
r = httpx.get(url)
data = json.loads(r.text)
stock = data[0]['hq']
df = pd.read_json(json.dumps(stock), orient="records")
df.columns = ['日期', '开盘价','收盘价','涨跌幅度', '涨跌百分比','最低价', '最高价', '成交股票手数', '成交金额(单位：万)', '换手率']

df = df.set_index("日期")
df.index = pd.to_datetime(df.index)
df

,开盘价,收盘价,涨跌幅度,涨跌百分比,最低价,最高价,成交股票手数,成交金额(单位：万),换手率
日期,,,,,,,,,
2022-07-22,35.55,35.59,0.13,0.37%,35.25,36.05,510333,181747.73,0.25%
2022-07-21,35.54,35.46,-0.39,-1.09%,35.37,35.83,466241,165613.77,0.23%
2022-07-20,35.94,35.85,0.14,0.39%,35.71,36.12,469689,168511.69,0.23%
2022-07-19,35.50,35.71,0.08,0.22%,35.20,35.85,537814,191044.84,0.26%
2022-07-18,35.15,35.63,0.76,2.18%,35.01,35.84,903080,320222.72,0.44%
...,...,...,...,...,...,...,...,...,...
2022-01-10,49.47,49.23,0.18,0.37%,48.91,50.00,570936,281814.50,0.28%
2022-01-07,48.14,49.05,0.90,1.87%,48.04,49.14,660737,321952.34,0.32%
2022-01-06,48.82,48.15,-0.90,-1.83%,48.10,49.39,681867,329982.56,0.33%


## 新浪财经基金净值数据

url：

http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20220101&dateto=20220723&page=1

接口是分页的形式，需要遍历 page。

```
'日期' => 'fbrq',
'单位净值（元）' => 'jjjz',
'累计净值（元）' => 'ljjz',
```

In [5]:
page = 1
total = pd.DataFrame([])

while True:
    url = f'http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20210101&dateto=20220723&page={page}'
    #print(f'fetch url: {url}')
    r = httpx.get(url)
    data = json.loads(r.text)
    result = data["result"]["data"]['data']
    if (len(result) == 0):
        break
    df = pd.read_json(json.dumps(result), orient="records")
    total = pd.concat([total, df])
    page = page + 1


total.columns = ['日期', '单位净值（元）', '累计净值（元）']
total = total.set_index('日期')
total.index = pd.to_datetime(total.index)
total

,单位净值（元）,累计净值（元）
日期,,
2022-07-22,2.7210,4.2510
2022-07-21,2.7123,4.2423
2022-07-20,2.7432,4.2732
2022-07-19,2.7116,4.2416
2022-07-18,2.7372,4.2672
...,...,...
2021-01-08,2.8864,4.4164
2021-01-07,2.9176,4.4476
2021-01-06,2.8316,4.3616


## 新浪财经

新浪财经网页地址：https://vip.stock.finance.sina.com.cn/mkt/#hqIndex

url地址：

```
https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQNodeData?page=1&num=80&sort=symbol&asc=1&node=hs_a&symbol=&_s_r_a=init
```

`node` 参数：hs_a 为沪深A股


In [6]:
url = "https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQNodeData?page=1&num=80&sort=symbol&asc=1&node=hs_a&symbol=&_s_r_a=init"
r = httpx.get(url)
data = json.loads(r.text)
df = pd.read_json(json.dumps(data), orient="records")
df = df.set_index("symbol")
df

,code,name,trade,pricechange,changepercent,buy,sell,settlement,open,high,low,volume,amount,ticktime,per,pb,mktcap,nmc,turnoverratio
symbol,,,,,,,,,,,,,,,,,,,
bj430047,430047,诺思兰德,13.30,-1.03,-7.188,13.28,13.30,14.33,14.20,14.33,13.30,898875,12512749,15:30:17,-66.500,16.432,3.427652e+05,1.985151e+05,0.60222
bj430090,430090,同辉信息,2.76,-0.01,-0.361,2.76,2.77,2.77,2.77,2.79,2.75,761134,2110263,15:30:17,14.634,1.778,5.501606e+04,3.875445e+04,0.54206
bj430139,430139,华岭股份,10.05,-0.43,-4.103,10.02,10.05,10.48,10.50,10.57,10.00,619468,6257185,15:30:17,25.125,2.649,2.681340e+05,1.252051e+05,0.49724
bj430198,430198,微创光电,4.75,0.00,0.000,4.67,4.75,4.75,4.66,4.75,4.65,25568,120876,15:30:17,13.971,1.637,7.664784e+04,3.083198e+04,0.03939
bj430300,430300,辰光医疗,6.43,-0.22,-3.308,6.40,6.43,6.65,6.71,6.71,6.35,1077723,7049547,15:30:17,18.371,2.583,5.375295e+04,3.024896e+04,2.29091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bj834765,834765,美之高,6.42,-0.03,-0.465,6.30,6.41,6.45,6.48,6.48,6.28,102980,661597,15:30:17,35.667,1.288,4.469157e+04,1.313449e+04,0.50336
bj835174,835174,五新隧装,7.83,-0.08,-1.011,7.83,7.85,7.91,7.88,7.92,7.83,52751,415040,15:30:17,6.418,1.259,7.047683e+04,4.139397e+04,0.09978
bj835179,835179,凯德石英,17.89,0.01,0.056,17.80,17.89,17.88,17.68,17.94,17.66,31827,567583,15:30:17,27.106,2.052,1.341750e+05,7.855452e+04,0.07248


**基金数据：**

url：

```
https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getFundNetData?page=2&num=40&sort=symbol&asc=1&node=open_fund&_s_r_a=page
```


In [7]:
url = "https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getFundNetData?page=1&num=80&sort=symbol&asc=1&node=open_fund&_s_r_a=page"
r = httpx.get(url)
data = json.loads(r.text)
df = pd.read_json(json.dumps(data), orient="records", dtype={'symbol': str})
df.columns = ["代码", "名称", "最新公布净值", "累计净值", "前单位净值", "净值增长率", "日期", "基金规模份额（亿份）"]
df = df.set_index('代码')
df

,名称,最新公布净值,累计净值,前单位净值,净值增长率,日期,基金规模份额（亿份）
代码,,,,,,,
000001,华夏成长混合,0.9630,3.5240,0.9700,-0.721649,2022-12-22,31.1160
000003,中海可转债债券A,0.8120,1.0220,0.8170,-0.611995,2022-12-22,0.2648
000004,中海可转债债券C,0.8030,1.0130,0.8070,-0.495663,2022-12-22,0.3233
000005,嘉实增强信用定期债券,1.0224,1.4552,1.0218,0.058720,2022-12-22,1.3095
000006,西部利得量化成长混合A,1.8233,2.1513,1.8466,-1.261778,2022-12-22,7.2720
...,...,...,...,...,...,...,...
000112,易方达纯债1年定期开放债券C,1.0510,1.4320,1.0500,0.095238,2022-12-22,0.0844
000116,嘉实丰益纯债定期债券,0.9988,1.4013,0.9986,0.020028,2022-12-22,3.1457
000117,广发轮动配置混合,2.4120,2.4120,2.3940,0.751880,2022-12-21,1.6062


**ETF数据：**

url:

```
https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getFundNetData?page=3&num=40&sort=symbol&asc=1&node=etf_jz_fund&_s_r_a=page
```

In [8]:
url = "https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getFundNetData?page=1&num=80&sort=symbol&asc=1&node=etf_jz_fund&_s_r_a=page"
r = httpx.get(url)
data = json.loads(r.text)
df = pd.read_json(json.dumps(data), orient="records", dtype={'symbol': str})
df.columns = ["代码", "名称", "最新公布净值", "累计净值", "前单位净值", "净值增长率", "日期", "基金规模份额（亿份）"]
df = df.set_index('代码')
df

,名称,最新公布净值,累计净值,前单位净值,净值增长率,日期,基金规模份额（亿份）
代码,,,,,,,
159001,易方达保证金货币A,0.5834,1.9260,0.0000,NaN,2022-12-22,0.0219
159003,招商保证金快线货币A,0.4335,1.5980,0.0000,NaN,2022-12-22,0.0168
159005,汇添富收益快钱货币A,0.5728,1.9540,0.0000,NaN,2022-12-22,0.0026
159601,华夏MSCI中国A50互联互通ETF,0.8000,0.8000,0.7980,0.250627,2022-12-22,58.1859
159602,南方MSCI中国A50互联互通ETF,0.8002,0.8002,0.7982,0.250564,2022-12-22,21.8654
...,...,...,...,...,...,...,...
159717,鹏华国证ESG300ETF,0.8487,0.8487,0.8468,0.224374,2022-12-22,0.6287
159718,平安中证港股医药ETF,0.8636,0.8636,0.8481,1.827615,2022-12-22,0.7883
159719,平安富时中国国企开放共赢ETF,1.0577,1.0577,1.0604,-0.254621,2022-12-22,0.3320
